# 1 Dataset and Preprocessing

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [6]:
filename = "shakespeare.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [164]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [163]:
n_chars = len(raw_text)
n_vocab = len(chars)

In [9]:
seq_length = 40
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)


In [10]:
# [n_samples, Tx, Tc]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
y = to_categorical(dataY)

In [11]:
print("Characters: ", n_chars)
print("Vocabulary: ", n_vocab)
print("Train Samples: ", n_patterns)

Characters:  93677
Vocabulary:  38
Train Samples:  93637


# 2 Language Modeling

In [12]:
# LSTM alpha=10

LSTM_cell = LSTM(120, input_shape=(X.shape[1], X.shape[2]))
densor = Dense(y.shape[1], activation='softmax')

model = Sequential()
#model.add(tf.keras.layers.Embedding(n_vocab, 40, input_length=y.shape[1]))
model.add(LSTM_cell)
#model.add(Dropout(0.2))
model.add(densor)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

## 2.1 Training

Recording all network weights to check improvement in loss at the end of every epoch

In [13]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
#model.fit(X, y, epochs=1000, batch_size=256, callbacks=callbacks_list)

In [15]:
tf.debugging.set_log_device_placement(True)

try:
    with tf.device('/device:GPU:0'): 
        # [n_samples, Tx, Tc]
        X = np.reshape(dataX, (n_patterns, seq_length, 1))
        # normalize
        X = X / float(n_vocab)
        y = to_categorical(dataY)
        LSTM_cell = LSTM(120, input_shape=(X.shape[1], X.shape[2]))
        densor = Dense(y.shape[1], activation='softmax')

        model = Sequential()
        #model.add(tf.keras.layers.Embedding(n_vocab, 40, input_length=y.shape[1]))
        model.add(LSTM_cell)
        #model.add(Dropout(0.2))
        model.add(densor)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])        
except RuntimeError as e:
    print(e)

saved_model = model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)


Epoch 1/1000
732/732 [==============================] - 9s 6ms/step - loss: 2.9753 - categorical_accuracy: 0.1714

Epoch 00001: loss improved from inf to 2.97527, saving model to weights-improvement-01-2.9753.hdf5
Epoch 2/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.7915 - categorical_accuracy: 0.2205

Epoch 00002: loss improved from 2.97527 to 2.79151, saving model to weights-improvement-02-2.7915.hdf5
Epoch 3/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.7256 - categorical_accuracy: 0.2304

Epoch 00003: loss improved from 2.79151 to 2.72561, saving model to weights-improvement-03-2.7256.hdf5
Epoch 4/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.6844 - categorical_accuracy: 0.2450

Epoch 00004: loss improved from 2.72561 to 2.68444, saving model to weights-improvement-04-2.6844.hdf5
Epoch 5/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.6421 - categorical_accuracy: 0.2587

Epoch 00005: loss

732/732 [==============================] - 4s 6ms/step - loss: 2.2342 - categorical_accuracy: 0.3519

Epoch 00038: loss improved from 2.24047 to 2.23424, saving model to weights-improvement-38-2.2342.hdf5
Epoch 39/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.2247 - categorical_accuracy: 0.3550

Epoch 00039: loss improved from 2.23424 to 2.22467, saving model to weights-improvement-39-2.2247.hdf5
Epoch 40/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.2174 - categorical_accuracy: 0.3565

Epoch 00040: loss improved from 2.22467 to 2.21742, saving model to weights-improvement-40-2.2174.hdf5
Epoch 41/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.2099 - categorical_accuracy: 0.3578

Epoch 00041: loss improved from 2.21742 to 2.20986, saving model to weights-improvement-41-2.2099.hdf5
Epoch 42/1000
732/732 [==============================] - 4s 6ms/step - loss: 2.2031 - categorical_accuracy: 0.3595

Epoch 00042: loss impr

732/732 [==============================] - 4s 6ms/step - loss: 2.0021 - categorical_accuracy: 0.4082

Epoch 00076: loss improved from 2.00573 to 2.00212, saving model to weights-improvement-76-2.0021.hdf5
Epoch 77/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.9980 - categorical_accuracy: 0.4103

Epoch 00077: loss improved from 2.00212 to 1.99801, saving model to weights-improvement-77-1.9980.hdf5
Epoch 78/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.9937 - categorical_accuracy: 0.4107

Epoch 00078: loss improved from 1.99801 to 1.99365, saving model to weights-improvement-78-1.9937.hdf5
Epoch 79/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.9880 - categorical_accuracy: 0.4124

Epoch 00079: loss improved from 1.99365 to 1.98800, saving model to weights-improvement-79-1.9880.hdf5
Epoch 80/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.9845 - categorical_accuracy: 0.4126

Epoch 00080: loss impr

732/732 [==============================] - 4s 6ms/step - loss: 1.8658 - categorical_accuracy: 0.4433

Epoch 00114: loss improved from 1.86937 to 1.86585, saving model to weights-improvement-114-1.8658.hdf5
Epoch 115/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.8633 - categorical_accuracy: 0.4442

Epoch 00115: loss improved from 1.86585 to 1.86330, saving model to weights-improvement-115-1.8633.hdf5
Epoch 116/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.8648 - categorical_accuracy: 0.4436

Epoch 00116: loss did not improve from 1.86330
Epoch 117/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.8580 - categorical_accuracy: 0.4467

Epoch 00117: loss improved from 1.86330 to 1.85798, saving model to weights-improvement-117-1.8580.hdf5
Epoch 118/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.8524 - categorical_accuracy: 0.4468

Epoch 00118: loss improved from 1.85798 to 1.85245, saving model to wei

732/732 [==============================] - 4s 6ms/step - loss: 1.7865 - categorical_accuracy: 0.4636

Epoch 00154: loss improved from 1.78682 to 1.78653, saving model to weights-improvement-154-1.7865.hdf5
Epoch 155/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.7824 - categorical_accuracy: 0.4650

Epoch 00155: loss improved from 1.78653 to 1.78243, saving model to weights-improvement-155-1.7824.hdf5
Epoch 156/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.7838 - categorical_accuracy: 0.4657

Epoch 00156: loss did not improve from 1.78243
Epoch 157/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.7777 - categorical_accuracy: 0.4671

Epoch 00157: loss improved from 1.78243 to 1.77768, saving model to weights-improvement-157-1.7777.hdf5
Epoch 158/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.7893 - categorical_accuracy: 0.4643

Epoch 00158: loss did not improve from 1.77768
Epoch 159/1000
732/732 [

...

732/732 [==============================] - 4s 6ms/step - loss: 1.5380 - categorical_accuracy: 0.5281

Epoch 00768: loss did not improve from 1.53021
Epoch 769/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5307 - categorical_accuracy: 0.5289

Epoch 00769: loss did not improve from 1.53021
Epoch 770/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5333 - categorical_accuracy: 0.5300

Epoch 00770: loss did not improve from 1.53021
Epoch 771/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5487 - categorical_accuracy: 0.5252

Epoch 00771: loss did not improve from 1.53021
Epoch 772/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5450 - categorical_accuracy: 0.5270

Epoch 00772: loss did not improve from 1.53021
Epoch 773/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5364 - categorical_accuracy: 0.5292

Epoch 00773: loss did not improve from 1.53021
Epoch 774/1000
732/732 [=======

732/732 [==============================] - 4s 6ms/step - loss: 1.5427 - categorical_accuracy: 0.5276

Epoch 00817: loss did not improve from 1.52246
Epoch 818/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5239 - categorical_accuracy: 0.5322

Epoch 00818: loss did not improve from 1.52246
Epoch 819/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5259 - categorical_accuracy: 0.5311

Epoch 00819: loss did not improve from 1.52246
Epoch 820/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5355 - categorical_accuracy: 0.5302

Epoch 00820: loss did not improve from 1.52246
Epoch 821/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5314 - categorical_accuracy: 0.5317

Epoch 00821: loss did not improve from 1.52246
Epoch 822/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5326 - categorical_accuracy: 0.5297

Epoch 00822: loss did not improve from 1.52246
Epoch 823/1000
732/732 [=======

732/732 [==============================] - 4s 6ms/step - loss: 1.5337 - categorical_accuracy: 0.5295

Epoch 00866: loss did not improve from 1.51974
Epoch 867/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5398 - categorical_accuracy: 0.5271

Epoch 00867: loss did not improve from 1.51974
Epoch 868/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5235 - categorical_accuracy: 0.5331

Epoch 00868: loss did not improve from 1.51974
Epoch 869/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5251 - categorical_accuracy: 0.5326

Epoch 00869: loss did not improve from 1.51974
Epoch 870/1000
732/732 [==============================] - 5s 6ms/step - loss: 1.5399 - categorical_accuracy: 0.5274

Epoch 00870: loss did not improve from 1.51974
Epoch 871/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5347 - categorical_accuracy: 0.5315

Epoch 00871: loss did not improve from 1.51974
Epoch 872/1000
732/732 [=======

732/732 [==============================] - 4s 6ms/step - loss: 1.5246 - categorical_accuracy: 0.5323

Epoch 00915: loss did not improve from 1.50796
Epoch 916/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5376 - categorical_accuracy: 0.5284

Epoch 00916: loss did not improve from 1.50796
Epoch 917/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5266 - categorical_accuracy: 0.5333

Epoch 00917: loss did not improve from 1.50796
Epoch 918/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5259 - categorical_accuracy: 0.5313

Epoch 00918: loss did not improve from 1.50796
Epoch 919/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5078 - categorical_accuracy: 0.5379

Epoch 00919: loss improved from 1.50796 to 1.50778, saving model to weights-improvement-919-1.5078.hdf5
Epoch 920/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5397 - categorical_accuracy: 0.5277

Epoch 00920: loss did


Epoch 00964: loss did not improve from 1.50778
Epoch 965/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5222 - categorical_accuracy: 0.5340

Epoch 00965: loss did not improve from 1.50778
Epoch 966/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5176 - categorical_accuracy: 0.5348

Epoch 00966: loss did not improve from 1.50778
Epoch 967/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5327 - categorical_accuracy: 0.5300

Epoch 00967: loss did not improve from 1.50778
Epoch 968/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5247 - categorical_accuracy: 0.5313

Epoch 00968: loss did not improve from 1.50778
Epoch 969/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5190 - categorical_accuracy: 0.5336

Epoch 00969: loss did not improve from 1.50778
Epoch 970/1000
732/732 [==============================] - 4s 6ms/step - loss: 1.5146 - categorical_accuracy: 0.5336

Epoch 00970: lo

In [16]:
model.save('model1.h5')

In [18]:
filepath2="2nd-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint2 = ModelCheckpoint(filepath2, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list2 = [checkpoint2]


try:
    with tf.device('/device:GPU:0'): 
        # [n_samples, Tx, Tc]
        X = np.reshape(dataX, (n_patterns, seq_length, 1))
        # normalize
        X = X / float(n_vocab)
        y = to_categorical(dataY)
        LSTM_cell_2 = LSTM(240, input_shape=(X.shape[1], X.shape[2]))
        densor_2 = Dense(y.shape[1], activation='softmax')

        model_2 = Sequential()
        #model.add(tf.keras.layers.Embedding(n_vocab, 40, input_length=y.shape[1]))
        model_2.add(LSTM_cell_2)
        #model.add(Dropout(0.2))
        model_2.add(densor_2)
        model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])        
except RuntimeError as e:
    print(e)

saved_model_2 = model_2.fit(X, y, epochs=1000, batch_size=256, callbacks=callbacks_list2)
model_2.save('model2.h5')

Epoch 1/1000
366/366 [==============================] - 6s 13ms/step - loss: 2.9892 - categorical_accuracy: 0.1654

Epoch 00001: loss improved from inf to 2.98924, saving model to 2nd-weights-improvement-01-2.9892.hdf5
Epoch 2/1000
366/366 [==============================] - 5s 12ms/step - loss: 2.8281 - categorical_accuracy: 0.2149

Epoch 00002: loss improved from 2.98924 to 2.82809, saving model to 2nd-weights-improvement-02-2.8281.hdf5
Epoch 3/1000
366/366 [==============================] - 5s 13ms/step - loss: 2.7391 - categorical_accuracy: 0.2263

Epoch 00003: loss improved from 2.82809 to 2.73908, saving model to 2nd-weights-improvement-03-2.7391.hdf5
Epoch 4/1000
366/366 [==============================] - 5s 12ms/step - loss: 2.6881 - categorical_accuracy: 0.2452

Epoch 00004: loss improved from 2.73908 to 2.68811, saving model to 2nd-weights-improvement-04-2.6881.hdf5
Epoch 5/1000
366/366 [==============================] - 5s 12ms/step - loss: 2.6558 - categorical_accuracy: 0.25

366/366 [==============================] - 5s 13ms/step - loss: 1.8344 - categorical_accuracy: 0.4550

Epoch 00038: loss improved from 1.86418 to 1.83439, saving model to 2nd-weights-improvement-38-1.8344.hdf5
Epoch 39/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.8109 - categorical_accuracy: 0.4603

Epoch 00039: loss improved from 1.83439 to 1.81088, saving model to 2nd-weights-improvement-39-1.8109.hdf5
Epoch 40/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.7824 - categorical_accuracy: 0.4691

Epoch 00040: loss improved from 1.81088 to 1.78236, saving model to 2nd-weights-improvement-40-1.7824.hdf5
Epoch 41/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.7585 - categorical_accuracy: 0.4755

Epoch 00041: loss improved from 1.78236 to 1.75847, saving model to 2nd-weights-improvement-41-1.7585.hdf5
Epoch 42/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.7349 - categorical_accuracy: 0.4836

E

366/366 [==============================] - 5s 13ms/step - loss: 1.1423 - categorical_accuracy: 0.6535

Epoch 00075: loss improved from 1.14880 to 1.14233, saving model to 2nd-weights-improvement-75-1.1423.hdf5
Epoch 76/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.1342 - categorical_accuracy: 0.6551

Epoch 00076: loss improved from 1.14233 to 1.13418, saving model to 2nd-weights-improvement-76-1.1342.hdf5
Epoch 77/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.1164 - categorical_accuracy: 0.6633

Epoch 00077: loss improved from 1.13418 to 1.11637, saving model to 2nd-weights-improvement-77-1.1164.hdf5
Epoch 78/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.1151 - categorical_accuracy: 0.6609

Epoch 00078: loss improved from 1.11637 to 1.11506, saving model to 2nd-weights-improvement-78-1.1151.hdf5
Epoch 79/1000
366/366 [==============================] - 5s 13ms/step - loss: 1.0956 - categorical_accuracy: 0.6683

E

366/366 [==============================] - 5s 13ms/step - loss: 0.8205 - categorical_accuracy: 0.7468

Epoch 00113: loss improved from 0.82546 to 0.82046, saving model to 2nd-weights-improvement-113-0.8205.hdf5
Epoch 114/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.7972 - categorical_accuracy: 0.7538

Epoch 00114: loss improved from 0.82046 to 0.79720, saving model to 2nd-weights-improvement-114-0.7972.hdf5
Epoch 115/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.8043 - categorical_accuracy: 0.7522

Epoch 00115: loss did not improve from 0.79720
Epoch 116/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.8017 - categorical_accuracy: 0.7521

Epoch 00116: loss did not improve from 0.79720
Epoch 117/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.7939 - categorical_accuracy: 0.7540

Epoch 00117: loss improved from 0.79720 to 0.79385, saving model to 2nd-weights-improvement-117-0.7939.hdf5
Epoch 1

366/366 [==============================] - 5s 13ms/step - loss: 0.6491 - categorical_accuracy: 0.7955

Epoch 00156: loss did not improve from 0.61983
Epoch 157/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.6357 - categorical_accuracy: 0.7999

Epoch 00157: loss did not improve from 0.61983
Epoch 158/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.6240 - categorical_accuracy: 0.8048

Epoch 00158: loss did not improve from 0.61983
Epoch 159/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.6077 - categorical_accuracy: 0.8101

Epoch 00159: loss improved from 0.61983 to 0.60771, saving model to 2nd-weights-improvement-159-0.6077.hdf5
Epoch 160/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.6199 - categorical_accuracy: 0.8052

Epoch 00160: loss did not improve from 0.60771
Epoch 161/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.6444 - categorical_accuracy: 0.7963

Epoch 00161

...

366/366 [==============================] - 5s 12ms/step - loss: 0.2597 - categorical_accuracy: 0.9168

Epoch 00784: loss did not improve from 0.20616
Epoch 785/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2962 - categorical_accuracy: 0.9029

Epoch 00785: loss did not improve from 0.20616
Epoch 786/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2953 - categorical_accuracy: 0.9041

Epoch 00786: loss did not improve from 0.20616
Epoch 787/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2656 - categorical_accuracy: 0.9122

Epoch 00787: loss did not improve from 0.20616
Epoch 788/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.2630 - categorical_accuracy: 0.9151

Epoch 00788: loss did not improve from 0.20616
Epoch 789/1000
366/366 [==============================] - 5s 13ms/step - loss: 0.2361 - categorical_accuracy: 0.9235

Epoch 00789: loss did not improve from 0.20616
Epoch 790/1000
366/366 [=

366/366 [==============================] - 5s 12ms/step - loss: 0.2461 - categorical_accuracy: 0.9201

Epoch 00833: loss did not improve from 0.19004
Epoch 834/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2323 - categorical_accuracy: 0.9259

Epoch 00834: loss did not improve from 0.19004
Epoch 835/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2192 - categorical_accuracy: 0.9305

Epoch 00835: loss did not improve from 0.19004
Epoch 836/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2874 - categorical_accuracy: 0.9054

Epoch 00836: loss did not improve from 0.19004
Epoch 837/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.3219 - categorical_accuracy: 0.8954

Epoch 00837: loss did not improve from 0.19004
Epoch 838/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.3068 - categorical_accuracy: 0.9014

Epoch 00838: loss did not improve from 0.19004
Epoch 839/1000
366/366 [=

366/366 [==============================] - 5s 12ms/step - loss: 0.2914 - categorical_accuracy: 0.9045

Epoch 00883: loss did not improve from 0.19004
Epoch 884/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2183 - categorical_accuracy: 0.9300

Epoch 00884: loss did not improve from 0.19004
Epoch 885/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2207 - categorical_accuracy: 0.9294

Epoch 00885: loss did not improve from 0.19004
Epoch 886/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2742 - categorical_accuracy: 0.9106

Epoch 00886: loss did not improve from 0.19004
Epoch 887/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.3384 - categorical_accuracy: 0.8901

Epoch 00887: loss did not improve from 0.19004
Epoch 888/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2670 - categorical_accuracy: 0.9137

Epoch 00888: loss did not improve from 0.19004
Epoch 889/1000
366/366 [=

366/366 [==============================] - 5s 12ms/step - loss: 0.2831 - categorical_accuracy: 0.9083

Epoch 00932: loss did not improve from 0.18278
Epoch 933/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2635 - categorical_accuracy: 0.9146

Epoch 00933: loss did not improve from 0.18278
Epoch 934/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2694 - categorical_accuracy: 0.9119

Epoch 00934: loss did not improve from 0.18278
Epoch 935/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2525 - categorical_accuracy: 0.9170

Epoch 00935: loss did not improve from 0.18278
Epoch 936/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2369 - categorical_accuracy: 0.9229

Epoch 00936: loss did not improve from 0.18278
Epoch 937/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2671 - categorical_accuracy: 0.9127

Epoch 00937: loss did not improve from 0.18278
Epoch 938/1000
366/366 [=

366/366 [==============================] - 5s 12ms/step - loss: 0.2614 - categorical_accuracy: 0.9144

Epoch 00982: loss did not improve from 0.18278
Epoch 983/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2053 - categorical_accuracy: 0.9342

Epoch 00983: loss did not improve from 0.18278
Epoch 984/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.1933 - categorical_accuracy: 0.9387

Epoch 00984: loss did not improve from 0.18278
Epoch 985/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2540 - categorical_accuracy: 0.9181

Epoch 00985: loss did not improve from 0.18278
Epoch 986/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.3055 - categorical_accuracy: 0.9016

Epoch 00986: loss did not improve from 0.18278
Epoch 987/1000
366/366 [==============================] - 5s 12ms/step - loss: 0.2654 - categorical_accuracy: 0.9140

Epoch 00987: loss did not improve from 0.18278
Epoch 988/1000
366/366 [=

In [17]:
filepath3="3rd-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint3 = ModelCheckpoint(filepath3, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list3 = [checkpoint3]


try:
    with tf.device('/device:GPU:0'): 
        # [n_samples, Tx, Tc]
        X = np.reshape(dataX, (n_patterns, seq_length, 1))
        # normalize
        X = X / float(n_vocab)
        y = to_categorical(dataY)
        LSTM_cell_3 = LSTM(600, input_shape=(X.shape[1], X.shape[2]))
        densor_3 = Dense(y.shape[1], activation='softmax')

        model_3 = Sequential()
        #model.add(tf.keras.layers.Embedding(n_vocab, 40, input_length=y.shape[1]))
        model_3.add(LSTM_cell_3)
        #model.add(Dropout(0.2))
        model_3.add(densor_3)
        model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])        
except RuntimeError as e:
    print(e)

saved_model_3 = model_3.fit(X, y, epochs=1000, batch_size=320, callbacks=callbacks_list3)
model_3.save('model3.h5')

Epoch 1/1000
293/293 [==============================] - 14s 41ms/step - loss: 2.9836 - categorical_accuracy: 0.1695

Epoch 00001: loss improved from inf to 2.98356, saving model to 3rd-weights-improvement-01-2.9836.hdf5
Epoch 2/1000
293/293 [==============================] - 12s 41ms/step - loss: 2.8055 - categorical_accuracy: 0.2154

Epoch 00002: loss improved from 2.98356 to 2.80554, saving model to 3rd-weights-improvement-02-2.8055.hdf5
Epoch 3/1000
293/293 [==============================] - 12s 41ms/step - loss: 2.7034 - categorical_accuracy: 0.2404

Epoch 00003: loss improved from 2.80554 to 2.70335, saving model to 3rd-weights-improvement-03-2.7034.hdf5
Epoch 4/1000
293/293 [==============================] - 12s 42ms/step - loss: 2.6529 - categorical_accuracy: 0.2545

Epoch 00004: loss improved from 2.70335 to 2.65286, saving model to 3rd-weights-improvement-04-2.6529.hdf5
Epoch 5/1000
293/293 [==============================] - 12s 42ms/step - loss: 2.6145 - categorical_accuracy:

293/293 [==============================] - 13s 43ms/step - loss: 0.2096 - categorical_accuracy: 0.9528

Epoch 00037: loss did not improve from 0.19476
Epoch 38/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.2006 - categorical_accuracy: 0.9546

Epoch 00038: loss did not improve from 0.19476
Epoch 39/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.1705 - categorical_accuracy: 0.9633

Epoch 00039: loss improved from 0.19476 to 0.17053, saving model to 3rd-weights-improvement-39-0.1705.hdf5
Epoch 40/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.1367 - categorical_accuracy: 0.9733

Epoch 00040: loss improved from 0.17053 to 0.13671, saving model to 3rd-weights-improvement-40-0.1367.hdf5
Epoch 41/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.1368 - categorical_accuracy: 0.9726

Epoch 00041: loss did not improve from 0.13671
Epoch 42/1000
293/293 [==============================] - 13s 43ms/ste

293/293 [==============================] - 13s 43ms/step - loss: 0.0039 - categorical_accuracy: 1.0000

Epoch 00080: loss did not improve from 0.00128
Epoch 81/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0033 - categorical_accuracy: 1.0000

Epoch 00081: loss did not improve from 0.00128
Epoch 82/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0030 - categorical_accuracy: 1.0000

Epoch 00082: loss did not improve from 0.00128
Epoch 83/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0025 - categorical_accuracy: 1.0000

Epoch 00083: loss did not improve from 0.00128
Epoch 84/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0021 - categorical_accuracy: 1.0000

Epoch 00084: loss did not improve from 0.00128
Epoch 85/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0018 - categorical_accuracy: 1.0000

Epoch 00085: loss did not improve from 0.00128
Epoch 86/1000
293/293 [=

293/293 [==============================] - 13s 43ms/step - loss: 0.0015 - categorical_accuracy: 1.0000

Epoch 00129: loss did not improve from 0.00128
Epoch 130/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0014 - categorical_accuracy: 1.0000

Epoch 00130: loss did not improve from 0.00128
Epoch 131/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0011 - categorical_accuracy: 1.0000

Epoch 00131: loss improved from 0.00128 to 0.00109, saving model to 3rd-weights-improvement-131-0.0011.hdf5
Epoch 132/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0010 - categorical_accuracy: 1.0000

Epoch 00132: loss improved from 0.00109 to 0.00103, saving model to 3rd-weights-improvement-132-0.0010.hdf5
Epoch 133/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.9460 - categorical_accuracy: 0.7219

Epoch 00133: loss did not improve from 0.00103
Epoch 134/1000
293/293 [==============================] - 13s 4

293/293 [==============================] - 13s 43ms/step - loss: 0.0013 - categorical_accuracy: 1.0000

Epoch 00177: loss did not improve from 0.00099
Epoch 178/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0012 - categorical_accuracy: 1.0000

Epoch 00178: loss did not improve from 0.00099
Epoch 179/1000
293/293 [==============================] - 13s 43ms/step - loss: 0.0010 - categorical_accuracy: 1.0000

Epoch 00179: loss did not improve from 0.00099
Epoch 180/1000
293/293 [==============================] - 13s 43ms/step - loss: 9.3362e-04 - categorical_accuracy: 1.0000

Epoch 00180: loss improved from 0.00099 to 0.00093, saving model to 3rd-weights-improvement-180-0.0009.hdf5
Epoch 181/1000
293/293 [==============================] - 13s 43ms/step - loss: 8.2044e-04 - categorical_accuracy: 1.0000

Epoch 00181: loss improved from 0.00093 to 0.00082, saving model to 3rd-weights-improvement-181-0.0008.hdf5
Epoch 182/1000
293/293 [==============================]

...

293/293 [==============================] - 13s 43ms/step - loss: 4.6925e-05 - categorical_accuracy: 1.0000

Epoch 00793: loss did not improve from 0.00003
Epoch 794/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.4595e-05 - categorical_accuracy: 1.0000

Epoch 00794: loss did not improve from 0.00003
Epoch 795/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.2069e-05 - categorical_accuracy: 1.0000

Epoch 00795: loss did not improve from 0.00003
Epoch 796/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.0142e-05 - categorical_accuracy: 1.0000

Epoch 00796: loss did not improve from 0.00003
Epoch 797/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.7525e-05 - categorical_accuracy: 1.0000

Epoch 00797: loss did not improve from 0.00003
Epoch 798/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.5501e-05 - categorical_accuracy: 1.0000

Epoch 00798: loss did not improve from 0.0

293/293 [==============================] - 13s 43ms/step - loss: 3.4718e-05 - categorical_accuracy: 1.0000

Epoch 00841: loss did not improve from 0.00003
Epoch 842/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.3922e-05 - categorical_accuracy: 1.0000

Epoch 00842: loss did not improve from 0.00003
Epoch 843/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.2709e-05 - categorical_accuracy: 1.0000

Epoch 00843: loss did not improve from 0.00003
Epoch 844/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.1769e-05 - categorical_accuracy: 1.0000

Epoch 00844: loss did not improve from 0.00003
Epoch 845/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.0009e-05 - categorical_accuracy: 1.0000

Epoch 00845: loss did not improve from 0.00003
Epoch 846/1000
293/293 [==============================] - 13s 43ms/step - loss: 2.9412e-05 - categorical_accuracy: 1.0000

Epoch 00846: loss did not improve from 0.0

293/293 [==============================] - 13s 43ms/step - loss: 3.9687e-05 - categorical_accuracy: 1.0000

Epoch 00887: loss did not improve from 0.00002
Epoch 888/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.4958e-05 - categorical_accuracy: 1.0000

Epoch 00888: loss did not improve from 0.00002
Epoch 889/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.3951e-05 - categorical_accuracy: 1.0000

Epoch 00889: loss did not improve from 0.00002
Epoch 890/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.3591e-05 - categorical_accuracy: 1.0000

Epoch 00890: loss did not improve from 0.00002
Epoch 891/1000
293/293 [==============================] - 13s 43ms/step - loss: 3.2385e-05 - categorical_accuracy: 1.0000

Epoch 00891: loss did not improve from 0.00002
Epoch 892/1000
293/293 [==============================] - 13s 43ms/step - loss: 2.7313e-05 - categorical_accuracy: 1.0000

Epoch 00892: loss did not improve from 0.0

293/293 [==============================] - 13s 43ms/step - loss: 6.1599e-05 - categorical_accuracy: 1.0000

Epoch 00934: loss did not improve from 0.00002
Epoch 935/1000
293/293 [==============================] - 13s 43ms/step - loss: 5.6300e-05 - categorical_accuracy: 1.0000

Epoch 00935: loss did not improve from 0.00002
Epoch 936/1000
293/293 [==============================] - 13s 43ms/step - loss: 5.1278e-05 - categorical_accuracy: 1.0000

Epoch 00936: loss did not improve from 0.00002
Epoch 937/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.9122e-05 - categorical_accuracy: 1.0000

Epoch 00937: loss did not improve from 0.00002
Epoch 938/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.5406e-05 - categorical_accuracy: 1.0000

Epoch 00938: loss did not improve from 0.00002
Epoch 939/1000
293/293 [==============================] - 13s 43ms/step - loss: 4.3486e-05 - categorical_accuracy: 1.0000

Epoch 00939: loss did not improve from 0.0

293/293 [==============================] - 13s 43ms/step - loss: 1.8464e-04 - categorical_accuracy: 1.0000

Epoch 00982: loss did not improve from 0.00002
Epoch 983/1000
293/293 [==============================] - 13s 43ms/step - loss: 1.6378e-04 - categorical_accuracy: 1.0000

Epoch 00983: loss did not improve from 0.00002
Epoch 984/1000
293/293 [==============================] - 13s 43ms/step - loss: 1.4639e-04 - categorical_accuracy: 1.0000

Epoch 00984: loss did not improve from 0.00002
Epoch 985/1000
293/293 [==============================] - 13s 43ms/step - loss: 1.3175e-04 - categorical_accuracy: 1.0000

Epoch 00985: loss did not improve from 0.00002
Epoch 986/1000
293/293 [==============================] - 13s 43ms/step - loss: 1.1846e-04 - categorical_accuracy: 1.0000

Epoch 00986: loss did not improve from 0.00002
Epoch 987/1000
293/293 [==============================] - 13s 43ms/step - loss: 1.0747e-04 - categorical_accuracy: 1.0000

Epoch 00987: loss did not improve from 0.0

In [19]:
filepath4="4th-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint4 = ModelCheckpoint(filepath3, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list4 = [checkpoint4]


try:
    with tf.device('/device:GPU:0'): 
        # [n_samples, Tx, Tc]
        X = np.reshape(dataX, (n_patterns, seq_length, 1))
        # normalize
        X = X / float(n_vocab)
        y = to_categorical(dataY)
        LSTM_cell_4 = LSTM(120, input_shape=(X.shape[1], X.shape[2]))
        densor_4 = Dense(y.shape[1], activation='softmax')

        model_4 = Sequential()
        model.add(tf.keras.layers.Embedding(n_vocab, 40, input_length=y.shape[1]))
        model_4.add(LSTM_cell_4)
        model.add(Dropout(0.2))
        model_4.add(densor_4)
        model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])        
except RuntimeError as e:
    print(e)

saved_model_4 = model_4.fit(X, y, epochs=1000, batch_size=320, callbacks=callbacks_list4)
model_4.save('model4.h5')

Epoch 1/1000
293/293 [==============================] - 3s 7ms/step - loss: 3.0065 - categorical_accuracy: 0.1643

Epoch 00001: loss improved from inf to 3.00653, saving model to 3rd-weights-improvement-01-3.0065.hdf5
Epoch 2/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.8691 - categorical_accuracy: 0.2008

Epoch 00002: loss improved from 3.00653 to 2.86910, saving model to 3rd-weights-improvement-02-2.8691.hdf5
Epoch 3/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.7837 - categorical_accuracy: 0.2205

Epoch 00003: loss improved from 2.86910 to 2.78370, saving model to 3rd-weights-improvement-03-2.7837.hdf5
Epoch 4/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.7461 - categorical_accuracy: 0.2259

Epoch 00004: loss improved from 2.78370 to 2.74609, saving model to 3rd-weights-improvement-04-2.7461.hdf5
Epoch 5/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.7205 - categorical_accuracy: 0.2298

E

293/293 [==============================] - 2s 6ms/step - loss: 2.3729 - categorical_accuracy: 0.3135

Epoch 00038: loss improved from 2.37981 to 2.37286, saving model to 3rd-weights-improvement-38-2.3729.hdf5
Epoch 39/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.3688 - categorical_accuracy: 0.3142

Epoch 00039: loss improved from 2.37286 to 2.36884, saving model to 3rd-weights-improvement-39-2.3688.hdf5
Epoch 40/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.3619 - categorical_accuracy: 0.3179

Epoch 00040: loss improved from 2.36884 to 2.36189, saving model to 3rd-weights-improvement-40-2.3619.hdf5
Epoch 41/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.3549 - categorical_accuracy: 0.3194

Epoch 00041: loss improved from 2.36189 to 2.35493, saving model to 3rd-weights-improvement-41-2.3549.hdf5
Epoch 42/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.3501 - categorical_accuracy: 0.3200

Epoch 

293/293 [==============================] - 2s 6ms/step - loss: 2.1938 - categorical_accuracy: 0.3598

Epoch 00075: loss improved from 2.19742 to 2.19377, saving model to 3rd-weights-improvement-75-2.1938.hdf5
Epoch 76/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.1903 - categorical_accuracy: 0.3602

Epoch 00076: loss improved from 2.19377 to 2.19031, saving model to 3rd-weights-improvement-76-2.1903.hdf5
Epoch 77/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.1861 - categorical_accuracy: 0.3611

Epoch 00077: loss improved from 2.19031 to 2.18614, saving model to 3rd-weights-improvement-77-2.1861.hdf5
Epoch 78/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.1821 - categorical_accuracy: 0.3619

Epoch 00078: loss improved from 2.18614 to 2.18215, saving model to 3rd-weights-improvement-78-2.1821.hdf5
Epoch 79/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.1797 - categorical_accuracy: 0.3617

Epoch 

293/293 [==============================] - 2s 6ms/step - loss: 2.0532 - categorical_accuracy: 0.3941

Epoch 00112: loss improved from 2.05672 to 2.05321, saving model to 3rd-weights-improvement-112-2.0532.hdf5
Epoch 113/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.0542 - categorical_accuracy: 0.3933

Epoch 00113: loss did not improve from 2.05321
Epoch 114/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.0482 - categorical_accuracy: 0.3956

Epoch 00114: loss improved from 2.05321 to 2.04824, saving model to 3rd-weights-improvement-114-2.0482.hdf5
Epoch 115/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.0422 - categorical_accuracy: 0.3965

Epoch 00115: loss improved from 2.04824 to 2.04216, saving model to 3rd-weights-improvement-115-2.0422.hdf5
Epoch 116/1000
293/293 [==============================] - 2s 6ms/step - loss: 2.0398 - categorical_accuracy: 0.3964

Epoch 00116: loss improved from 2.04216 to 2.03982, saving 

293/293 [==============================] - 2s 6ms/step - loss: 1.9435 - categorical_accuracy: 0.4227

Epoch 00150: loss improved from 1.94697 to 1.94350, saving model to 3rd-weights-improvement-150-1.9435.hdf5
Epoch 151/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.9403 - categorical_accuracy: 0.4233

Epoch 00151: loss improved from 1.94350 to 1.94032, saving model to 3rd-weights-improvement-151-1.9403.hdf5
Epoch 152/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.9417 - categorical_accuracy: 0.4222

Epoch 00152: loss did not improve from 1.94032
Epoch 153/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.9372 - categorical_accuracy: 0.4228

Epoch 00153: loss improved from 1.94032 to 1.93717, saving model to 3rd-weights-improvement-153-1.9372.hdf5
Epoch 154/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.9318 - categorical_accuracy: 0.4251

Epoch 00154: loss improved from 1.93717 to 1.93176, saving 

...

293/293 [==============================] - 2s 6ms/step - loss: 1.5739 - categorical_accuracy: 0.5211

Epoch 00802: loss did not improve from 1.57030
Epoch 803/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.6014 - categorical_accuracy: 0.5135

Epoch 00803: loss did not improve from 1.57030
Epoch 804/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5783 - categorical_accuracy: 0.5209

Epoch 00804: loss did not improve from 1.57030
Epoch 805/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5841 - categorical_accuracy: 0.5175

Epoch 00805: loss did not improve from 1.57030
Epoch 806/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5779 - categorical_accuracy: 0.5196

Epoch 00806: loss did not improve from 1.57030
Epoch 807/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5838 - categorical_accuracy: 0.5194

Epoch 00807: loss did not improve from 1.57030
Epoch 808/1000
293/293 [=======

293/293 [==============================] - 2s 6ms/step - loss: 1.6321 - categorical_accuracy: 0.5047

Epoch 00851: loss did not improve from 1.56345
Epoch 852/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5827 - categorical_accuracy: 0.5191

Epoch 00852: loss did not improve from 1.56345
Epoch 853/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5649 - categorical_accuracy: 0.5230

Epoch 00853: loss did not improve from 1.56345
Epoch 854/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5703 - categorical_accuracy: 0.5224

Epoch 00854: loss did not improve from 1.56345
Epoch 855/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5876 - categorical_accuracy: 0.5176

Epoch 00855: loss did not improve from 1.56345
Epoch 856/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5781 - categorical_accuracy: 0.5201

Epoch 00856: loss did not improve from 1.56345
Epoch 857/1000
293/293 [=======

293/293 [==============================] - 2s 6ms/step - loss: 1.5648 - categorical_accuracy: 0.5253

Epoch 00900: loss did not improve from 1.55771
Epoch 901/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5620 - categorical_accuracy: 0.5251

Epoch 00901: loss did not improve from 1.55771
Epoch 902/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5616 - categorical_accuracy: 0.5248

Epoch 00902: loss did not improve from 1.55771
Epoch 903/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5669 - categorical_accuracy: 0.5225

Epoch 00903: loss did not improve from 1.55771
Epoch 904/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5798 - categorical_accuracy: 0.5194

Epoch 00904: loss did not improve from 1.55771
Epoch 905/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5651 - categorical_accuracy: 0.5231

Epoch 00905: loss did not improve from 1.55771
Epoch 906/1000
293/293 [=======

293/293 [==============================] - 2s 6ms/step - loss: 1.5630 - categorical_accuracy: 0.5246

Epoch 00949: loss did not improve from 1.54799
Epoch 950/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5865 - categorical_accuracy: 0.5175

Epoch 00950: loss did not improve from 1.54799
Epoch 951/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5704 - categorical_accuracy: 0.5219

Epoch 00951: loss did not improve from 1.54799
Epoch 952/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.6895 - categorical_accuracy: 0.4918

Epoch 00952: loss did not improve from 1.54799
Epoch 953/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5632 - categorical_accuracy: 0.5241

Epoch 00953: loss did not improve from 1.54799
Epoch 954/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5522 - categorical_accuracy: 0.5255

Epoch 00954: loss did not improve from 1.54799
Epoch 955/1000
293/293 [=======

293/293 [==============================] - 2s 6ms/step - loss: 1.5560 - categorical_accuracy: 0.5253

Epoch 00998: loss did not improve from 1.54710
Epoch 999/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5588 - categorical_accuracy: 0.5256

Epoch 00999: loss did not improve from 1.54710
Epoch 1000/1000
293/293 [==============================] - 2s 6ms/step - loss: 1.5644 - categorical_accuracy: 0.5231

Epoch 01000: loss did not improve from 1.54710


# 3 Predicting and Sampling

Here one may run the model without previous training as it is already trained and saved in 'model3.h5'

In [217]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

model_gen = tf.keras.models.load_model('model3.h5')
#filename_gen = "3rd-weights-improvement-978-1.5471.hdf5"
#model_gen.load_weights(filename_gen)
model_gen.compile(loss='categorical_crossentropy', optimizer='adam')

In [106]:
import sys

def gen_text(seed, length):
    pattern = [char_to_int[value] for value in seed]
    print("\"", seed, "\"")
    for i in range(length):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model_gen.predict(x, verbose=0)
        index = np.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        

Sampling:

In [107]:
start_position = np.random.randint(0,  len(dataX)-1)

seed_text = "".join([int_to_char[value] for value in dataX[start_position]])

gen_text(seed_text, 400)

"  your fair no painting set,
i found (or  "
thought i found) you did exceed,
that barren tender of a poet's debt:
and therefore have i slept in your report,
that you your self being extant well might show,
how far a modern quill doth come too short,
speaking of worth, what worth in you doth grow.  
this silence for my sin you did impute,
which shall be most my glory being dumb,
for i impair not beauty being mute,
when others would give life

# 4 Evaluation

## 4.1 400-char sequence with generated output

In [108]:
seed_text = "love to hear her speak, yet well i know,"

gen_text(seed_text, 400)

" love to hear her speak, yet well i know, "

that music hath a far more pleasing sound:
i grant i never saw a goddess go,
my mistress when she walks treads on the ground.
and yet by heaven i think my love as rare,
as any she belied with false compare.



thou art as tyrannous, so as thou art,  
as those whose beauties proudly make them cruel;
for well thou know'st to my dear doting heart
thou art the fairest and most precious jewel.
yet in 

## 4.2 Perplexity of the model on the sample test data in hold-out.txt:

In [218]:
filename_test = "hold-out.txt"
raw_text_test = open(filename_test, 'r', encoding='utf-8').read()
raw_text_test = raw_text_test.lower()

chars_test = sorted(list(set(raw_text_test)))
char_to_int_test = dict((c, i) for i, c in enumerate(chars_test))

n_chars_test = len(raw_text_test)
n_vocab_test = len(chars_test)

dataX_test = []
dataY_test = []
for i in range(0, n_chars_test - seq_length, 1):
    seq_in_test = raw_text_test[i:i + seq_length]
    seq_out_test = raw_text_test[i + seq_length]
    dataX_test.append([char_to_int[char] for char in seq_in_test])
    dataY_test.append(char_to_int[seq_out_test])
test_n_patterns = len(dataX_test)

# [n_samples, Tx, Tc]
X_test = np.reshape(dataX_test, (test_n_patterns, seq_length, 1))
# normalize
X_test = X_test / float(n_vocab)
y_test = to_categorical(dataY_test, 38)

In [231]:
test_loss = model_gen.evaluate(X_test, y_test)

18/18 [==============================] - 0s 18ms/step - loss: 11.8402


In [237]:
test_perplexity = tf.exp(test_loss)

In [240]:
test_perplexity.numpy()

138712.2